In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# get info on dataframe
application_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   EIN                     34299 non-null  int64 
 1   NAME                    34299 non-null  object
 2   APPLICATION_TYPE        34299 non-null  object
 3   AFFILIATION             34299 non-null  object
 4   CLASSIFICATION          34299 non-null  object
 5   USE_CASE                34299 non-null  object
 6   ORGANIZATION            34299 non-null  object
 7   STATUS                  34299 non-null  int64 
 8   INCOME_AMT              34299 non-null  object
 9   SPECIAL_CONSIDERATIONS  34299 non-null  object
 10  ASK_AMT                 34299 non-null  int64 
 11  IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 3.1+ MB


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN', 'NAME'], axis = 1, inplace = True)

In [4]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

#create function that takes a pandas series and an integer threshold value as arguments
def replace_other(pandas_series : pd.Series, threshold : int):

    #determine values to be replaced with vvalue counts less than determined threshold
    counts = pandas_series.value_counts()
    application_types_to_replace = [index for index, count in counts.items() if count < threshold]


    # Replace in dataframe
    for each in application_types_to_replace:
        pandas_series.replace(each,"Other", inplace=True)

    return pandas_series

# replace all application types with less than 100 value_counts with 'Other'
replace_other(application_df['APPLICATION_TYPE'], 100)

# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
replace_other(application_df['CLASSIFICATION'], 1000)

0        C1000
1        C2000
2        C3000
3        C2000
4        C1000
         ...  
34294    C1000
34295    C3000
34296    C2000
34297    C3000
34298    C1000
Name: CLASSIFICATION, Length: 34299, dtype: object

In [5]:
# Convert categorical data to numeric with `pd.get_dummies`
def get_dummies_from_df(dataframe : pd.DataFrame):

    #create copy of dataframe to work with
    dataframe_copy = dataframe.copy()

    #identify the categorical variables 
    categorical_series = dataframe_copy.select_dtypes(include='object').columns

    dummies = pd.get_dummies(dataframe_copy[categorical_series], dtype = int)
    
    dataframe_copy.drop(categorical_series, axis =1, inplace = True)
    dataframe_copy = pd.concat([dataframe_copy, dummies], axis = 1)

    return dataframe_copy

application_df_copy = get_dummies_from_df(application_df)

In [6]:
# Split our preprocessed data into our features and target arrays
y = application_df_copy['IS_SUCCESSFUL']
X = application_df_copy.drop('IS_SUCCESSFUL', axis = 1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test,y_train, y_test = train_test_split(X, y, random_state=1)

In [7]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [24]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):

    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=44))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 10)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=100,
            step=2),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [25]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

In [26]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 00m 40s]
val_accuracy: 0.7329446077346802

Best val_accuracy So Far: 0.7332944869995117
Total elapsed time: 00h 18m 40s


In [27]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'sigmoid',
 'first_units': 9,
 'num_layers': 8,
 'units_0': 85,
 'units_1': 19,
 'units_2': 79,
 'units_3': 87,
 'units_4': 43,
 'units_5': 95,
 'units_6': 59,
 'units_7': 67,
 'units_8': 5,
 'units_9': 57,
 'tuner/epochs': 7,
 'tuner/initial_epoch': 3,
 'tuner/bracket': 2,
 'tuner/round': 1,
 'tuner/trial_id': '0037'}

In [28]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - 3ms/step - accuracy: 0.7333 - loss: 0.5735
Loss: 0.5734991431236267, Accuracy: 0.7332944869995117


In [29]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_of_features = X_train_scaled.shape[1]

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=9, activation="sigmoid", input_dim=num_of_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=85, activation="sigmoid"))

# third hidden layer
nn.add(tf.keras.layers.Dense(units=19, activation="sigmoid"))

# fourth hidden layer
nn.add(tf.keras.layers.Dense(units=79, activation="sigmoid"))

# fifth hidden layer
nn.add(tf.keras.layers.Dense(units=87, activation="sigmoid"))

# sixth hidden layer
nn.add(tf.keras.layers.Dense(units=43, activation="sigmoid"))

# seventh hidden layer
nn.add(tf.keras.layers.Dense(units=95, activation="sigmoid"))

# eighth hidden layer
nn.add(tf.keras.layers.Dense(units=59, activation="sigmoid"))

# ninth hidden layer
nn.add(tf.keras.layers.Dense(units=67, activation="sigmoid"))

# ninth hidden layer
nn.add(tf.keras.layers.Dense(units=5, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=57, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_10 (Dense)                │ (None, 9)              │           405 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 85)             │           850 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 19)             │         1,634 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 79)             │         1,580 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 87)             │         6,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 43)             │         3,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 95)             │         4,180 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 59)             │         5,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 67)             │         4,020 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 5)              │           340 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 57)             │           342 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 1)              │            58 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,817 (116.47 KB)

 Trainable params: 29,817 (116.47 KB)

 Non-trainable params: 0 (0.00 B)

In [30]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [31]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs = 100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.5198 - loss: 0.6974
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5366 - loss: 0.6910
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5336 - loss: 0.6913
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5270 - loss: 0.6918
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5353 - loss: 0.6908
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5306 - loss: 0.6917
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5238 - loss: 0.6925
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5380 - loss: 0.6914
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5366 - loss: 0.6911
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5291 - loss: 0.6919
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5385 - loss: 0.6910
Epoch 12/100
804/804 ━━━━━━━━━━━━━━━━━━━━

In [32]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - 2ms/step - accuracy: 0.5292 - loss: 0.6930
Loss: 0.6930066347122192, Accuracy: 0.5292128324508667


In [12]:
# Export our model to HDF5 file
nn.save('Models/AlphabetSoupCharity_Optimization.h5')